## Music Recommendation System for Spotify

In [1]:
#lets import the required packages

import pandas as pd
import seaborn as sns
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
import pickle


In [2]:
#load the data set

df = pd.read_csv(r"E:\DOWNLOADS\GOD\Spotify music recommendations\spotify_millsongdata.csv")

In [3]:
#check the data frame
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
#shape
df.shape

(57650, 4)

In [5]:
#check null values
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
#drop the link column
df = df.sample(10000).drop('link', axis = 1).reset_index(drop = True)

In [7]:
#lets view the dataframe
df.head()

,artist,song,text
0,Lynyrd Skynyrd,Travelin' Man,"I was born a travelin' man, that's all Ill eve..."
1,Waterboys,Meet Me At The Station,Well if I get to heaven before you do \r\nI w...
2,Out Of Eden,Confused,I've been through so many changes I don't know...
3,Don Henley,I Can't Stand Still,I know you love me \r\nYou know I love you to...
4,Ween,Pretty Girl,"There's a sweater wearin', coat totin', boot f..."


#### Text cleaning and preprocessing


In [8]:
#Text cleaning
df['text'] = df['text'].str.lower().replace(r'^\w\s','').replace(r'\n', ' ', regex = True)

In [9]:
df

,artist,song,text
0,Lynyrd Skynyrd,Travelin' Man,"i was born a travelin' man, that's all ill eve..."
1,Waterboys,Meet Me At The Station,well if i get to heaven before you do \r i wi...
2,Out Of Eden,Confused,i've been through so many changes i don't know...
3,Don Henley,I Can't Stand Still,i know you love me \r you know i love you too...
4,Ween,Pretty Girl,"there's a sweater wearin', coat totin', boot f..."
...,...,...,...
9995,Kiss,Silver Spoon,"i ain't lookin' like nobody's fool, save it fo..."
9996,Lou Reed,Beginning Of A Great Adventure,it might be fun to have a kid that i could kic...
9997,Judas Priest,Take On The World,"you got to leave your seat, you gotta get up u..."
9998,Lionel Richie,Into You Deep,hey you \r where are you going \r you're not...


In [27]:
#create a funtion token
def token(txt):
        token = nltk.word_tokenize(txt)
        a = [stemmer.stem(w) for w in token]
        return " ".join(a)

In [11]:
#checking
token("your are awesome")

'your are awesom'

In [12]:
#applying token function to the text column
df["text"].apply(lambda x : token(x))

0       i wa born a travelin ' man , that 's all ill e...
1       well if i get to heaven befor you do i will me...
2       i 've been through so mani chang i do n't know...
3       i know you love me you know i love you too you...
4       there 's a sweater wearin ' , coat totin ' , b...
                              ...                        
9995    i ai n't lookin ' like nobodi 's fool , save i...
9996    it might be fun to have a kid that i could kic...
9997    you got to leav your seat , you got ta get up ...
9998    hey you where are you go you 're not leav me t...
9999    i pictur us togeth frame in gold leaf on the m...
Name: text, Length: 10000, dtype: object

In [28]:
# import packages
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
tfid = TfidfVectorizer(analyzer = 'word', stop_words = 'english')

In [15]:
matrix = tfid.fit_transform(df['text'])

In [16]:
similar = cosine_similarity(matrix)

In [20]:
similar[0]

array([1.00000000e+00, 5.76125706e-03, 9.74708196e-03, ...,
       1.04238936e-03, 4.69293689e-03, 6.07024994e-04])

### Recommender Function

In [22]:
def recommender(song_name):
    
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])),reverse = True ,  key = lambda x:x[1])
    song = []
    for s_id in distance[1:21]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [24]:
pickle.dump(similar, open("similarity","wb"))

In [25]:
pickle.dump(df, open("df", "wb"))

# Testing the recommendation system with simple GUI

In [ ]:
import tkinter as tk
from tkinter import ttk
import pickle
import pandas as pd

# Load data and similarity matrix
similar = pickle.load(open("similarity", "rb"))
df = pickle.load(open("df", "rb"))

class MusicRecommenderApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Music Recommender System")

        # Dropdown to select a song
        self.song_var = tk.StringVar()
        self.song_dropdown = ttk.Combobox(root, textvariable=self.song_var)
        self.song_dropdown['values'] = df['song'].tolist()
        self.song_dropdown.set("Select a Song")
        self.song_dropdown.grid(row=0, column=0, padx=10, pady=10)

        # Button to get recommendations
        self.recommend_button = tk.Button(root, text="Get Recommendations", command=self.get_recommendations)
        self.recommend_button.grid(row=0, column=1, padx=10, pady=10)

        # Recommended songs listbox
        self.recommended_songs_listbox = tk.Listbox(root, height=10, width=50)
        self.recommended_songs_listbox.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

        # Display selected song details
        self.selected_song_label = tk.Label(root, text="Selected Song Details:")
        self.selected_song_label.grid(row=2, column=0, columnspan=2, pady=10)

    def get_recommendations(self):
        selected_song = self.song_var.get()
        if selected_song:
            idx = df[df['song'] == selected_song].index[0]
            distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x: x[1])
            recommendations = [df.iloc[s_id[0]].song for s_id in distance[1:21]]

            # Display recommended songs in the listbox
            self.recommended_songs_listbox.delete(0, tk.END)
            for i, rec_song in enumerate(recommendations, start=1):
                self.recommended_songs_listbox.insert(tk.END, f"{i}. {rec_song}")

            # Display selected song details
            selected_song_details = df[df['song'] == selected_song].iloc[0]
            details_text = f"Artist: {selected_song_details['artist']}\nSong Text: {selected_song_details['text']}"
            self.selected_song_label.config(text=details_text)
        else:
            self.selected_song_label.config(text="Please select a song.")

if __name__ == "__main__":
    root = tk.Tk()
    app = MusicRecommenderApp(root)
    root.mainloop()